# Initialization

In [1]:
import matplotlib.pyplot as plt
import xarray as xr

plt.rcParams['figure.figsize'] = [16, 3]

import sys
sys.path.append('/mnt/lustre/IAM851/jm1667/psc/python')
import psc

import logging
# logging.basicConfig(level=logging.DEBUG)

%config InlineBackend.figure_format = 'retina'
engine = "pscadios2"
species_names = ['e', 'i']

In [2]:
case = "case1-ion"
B = 10
res = 512
mods = ["", "_v-1", '_v0', '_dup', '_T0', '_v2'][0]

path = f"/mnt/lustre/IAM851/jm1667/psc-runs/{case}/trials/B{B}_n{res}{mods}/"
length = {1:(1., .03, .03), 10:(1., .02, .02), .1:(1., .18, .18)}[B]
wholeSlice = slice(-length[1]/2, length[1]/2)
centerSlice = {10:slice(-.001, .001), 1:slice(-.003, .003), 0.1:slice(-.03, .03)}[B]

In [24]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 124

# read fields_every, moments_every from params file
with open(path + "params_record.txt") as records:
    fields_every = 200  # default value
    moments_every = 200  # default value
    for line in records:
        if line.startswith("fields_every"):
            fields_every = int(line.split()[1])
        elif line.startswith("moments_every"):
            moments_every = int(line.split()[1])

assert(fields_every == moments_every)

# check what the highest achieved timestep was
import os
bpfiles = [fname for fname in os.listdir(path) if fname[-2:] == "bp"]
nsteps = max(int(fname.split(".")[1]) for fname in bpfiles)

stepsPerFrame = nsteps // nframes
stepsPerFrame -= stepsPerFrame % fields_every

print(f"nsteps in sim: {nsteps}")
print(f"nframes in animation = {nframes}")
print(f"steps per frame: {stepsPerFrame}")
print(f"nsteps used: {stepsPerFrame * nframes}")
print(f"directory to save in: {case}/B{B}_n{res}{mods}_{nframes}x{stepsPerFrame}")

nsteps in sim: 436000
nframes in animation = 124
steps per frame: 3500
nsteps used: 434000
directory to save in: case1-ion/B10_n512_124x3500


# Animation
from https://stackoverflow.com/questions/18743673/show-consecutive-images-arrays-with-imshow-as-repeating-animation-in-python
and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

In [25]:
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

def openFile(fileName, step):
    return xr.open_dataset(path + f"{fileName}.{str(step).rjust(9,'0')}.bp", length=length, engine=engine, species_names=species_names)

def prepData(data):
    return data[0,:,:].transpose()

class DataLoader:
    def __init__(self):
        self.rGrid = None
        self.horGrid = None
        self.verGrid = None
    def getDataAndTime(self, frameIdx):
        if frameIdx == 0 and param.skipFirst:
            file = openFile(param.fileName, {'pfd':fields_every, 'pfd_moments':moments_every}[param.fileName])
        else:
            file = openFile(param.fileName, frameIdx*stepsPerFrame)

        if isinstance(param.varName, list):
            if param.combine == "magnitude":
                rawData = sum(prepData(file.__getattr__(var)) ** 2 for var in param.varName) ** .5
            elif param.combine == "sum":
                rawData = sum(prepData(file.__getattr__(var)) for var in param.varName)
            else:
                rawHor = prepData(file.__getattr__(param.varName[0]))
                rawVer = prepData(file.__getattr__(param.varName[1]))
                if self.rGrid is None:
                    self.rGrid = (rawHor.y **2 + rawHor.z**2)**.5
                    self.horGrid = rawHor.y
                    self.verGrid = rawHor.z
                if param.combine == 'radial':
                    rawData = (rawHor * self.horGrid + rawVer * self.verGrid) / self.rGrid
                elif param.combine == 'azimuthal':
                    rawData = (-rawHor * self.verGrid + rawVer * self.horGrid) / self.rGrid
                rawData = rawData.fillna(0)
        else:
            rawData = prepData(file.__getattr__(param.varName))
        return param.coef * rawData, file._attrs['time']
    def loadData(self):
        return [list(x) for x in zip(*[self.getDataAndTime(idx) for idx in range(nframes)])]

def setTitle(ax, param, time):
    ax.set_title(viewAdj + param.title + " (t={:.3f})".format(time))

class ParamMetadata:
    def __init__(self, title, vmin, vmax, colors, fileName, varName, coef=1, skipFirst=False, combine="magnitude"):
        self.title = title
        self.vmin = vmin
        self.vmax = vmax
        self.colors = colors
        self.fileName = fileName
        self.varName = varName
        self.coef = coef
        self.skipFirst = skipFirst
        self.combine = combine

In [36]:
paramId = 1
param = [
        ParamMetadata('Electron Density', 0, None, 'inferno', 'pfd_moments', 'rho_e', coef=-1),
        ParamMetadata('Ion Density', 0, None, 'inferno', 'pfd_moments', 'rho_i'),
        ParamMetadata('Y-ial E', -.0005, .0005, 'RdBu', 'pfd', 'ey_ec'),
        ParamMetadata('Y-ial B', -3e-8, 3e-8, 'RdBu', 'pfd', 'hy_fc', skipFirst=True),
        ParamMetadata('X-ial B', -B*1.2, B*1.2, 'RdBu', 'pfd', 'hx_fc'),
        ParamMetadata('Y-ial J', -.0005, .0005, 'RdBu', 'pfd', 'jy_ec', skipFirst=True),
        ParamMetadata('Radial J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='radial'),
        ParamMetadata('Azimuthal J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='azimuthal'),
        ParamMetadata('Electron Temperature', 0, None, 'inferno', 'pfd_moments', ['tyy_e', 'tzz_e'], skipFirst=False, combine='sum'),
        ][paramId]
print(f"quantity: {param.title}")

quantity: Ion Density


In [37]:
# load data
datas, times = DataLoader().loadData()

In [42]:
# slice data
activeSlice = [wholeSlice, centerSlice][1]

slicedDatas = [data.sel(y=activeSlice, z=activeSlice) for data in datas]
viewAdj = 'Central ' if activeSlice==centerSlice else ''
print(f"view: {viewAdj}= {activeSlice}")

vmax = param.vmax if not param.vmax is None else max(np.nanquantile(data.values, 1) for data in slicedDatas)
vmin = param.vmin if not param.vmin is None else min(np.nanquantile(data.values, 0) for data in slicedDatas)
if param.vmax is None and param.vmin is None:
    vmax = max(vmax, -vmin)
    vmin = -vmax

rGrid = (slicedDatas[0].y **2 + slicedDatas[0].z**2)**.5

view: Central = slice(-0.001, 0.001, None)


In [43]:
# view t=0
%matplotlib ipympl
fig, ax = plt.subplots()

im = ax.imshow(slicedDatas[0], cmap=param.colors, vmin=vmin, vmax=vmax, origin='lower', extent=(activeSlice.start, activeSlice.stop, activeSlice.start, activeSlice.stop))
ax.set_xlabel('y')
ax.set_ylabel('z')
setTitle(ax, param, times[0])
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
fig.colorbar(im, ax=ax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
# make movie
%matplotlib ipympl

def updateIm(frameIdx):
    im.set_array(slicedDatas[frameIdx])
    setTitle(ax, param, times[frameIdx])
    return [im]

anim = animation.FuncAnimation(fig, updateIm, interval=30, frames=nframes, repeat=False, blit=True)
HTML(anim.to_html5_video())

## Stability Plot

In [45]:
%matplotlib ipympl
def norm(x):
    return xr.apply_ufunc(np.linalg.norm, x, input_core_dims=[['y', 'z']])

normsOfDiffs = [norm(data - slicedDatas[0]) for data in slicedDatas]

plt.xlabel("Time")
plt.ylabel("2-Norm of Difference")
plt.title("Stability of " + viewAdj + param.title)

plt.plot(times, normsOfDiffs)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Radial Dependence

In [15]:
import matplotlib

maxR = max(activeSlice.start, activeSlice.stop) * 2**.5
rStep = length[1] / 100

def getMeanAndStd(data, r):
    rslice = data.where((r <= rGrid) & (rGrid < r + rStep))
    return rslice.mean().item(), rslice.std().item()
quantiles = np.linspace(0, 1, 11)
def getPercentiles(data, r):
    rslice = data.where((r <= rGrid) & (rGrid < r + rStep))
    return tuple(np.nanquantile(rslice.values, p) for p in quantiles)

rs = np.arange(0, maxR, rStep)
def getMeansAndStds(data):
    return tuple(zip(*[getMeanAndStd(data, r) for r in rs]))
def getPercentileses(data):
    return tuple(zip(*[getPercentiles(data, r) for r in rs]))

In [19]:
dataIdx = -1

### Mean & Error

In [20]:
means, stds = getMeansAndStds(slicedDatas[dataIdx])
means0, stds0 = getMeansAndStds(slicedDatas[0])

In [21]:
%matplotlib ipympl

plt.xlabel("Distance from Axis")
plt.ylabel(param.title)
plt.title("Mean " + param.title + " vs Radius (t={:.2f})".format(times[dataIdx]))

plt.errorbar(rs, means0, yerr=stds0, errorevery=(1,2), color="blue", ecolor="lightskyblue", elinewidth=1, capsize=1.5, label="t={:.2f}".format(0))
plt.errorbar(rs, means, yerr=stds, errorevery=(0, 2), color="red", ecolor="lightcoral", elinewidth=1, capsize=1.5, label="t={:.2f}".format(times[dataIdx]))
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Percentiles

In [50]:
percentiles = getPercentileses(slicedDatas[dataIdx])

In [51]:
%matplotlib ipympl

plt.xlabel("Distance from Axis, $\\rho$")
plt.ylabel(param.title)
plt.title(f"{len(quantiles)} Percentiles of " + param.title + " vs $\\rho$ (t={:.2f})".format(times[dataIdx]))

cmap = matplotlib.cm.get_cmap('rainbow')

labels = ["{:.0f}%".format(q*100) for q in quantiles]
colors = [cmap(2*abs(.5-q)) for q in quantiles]

for perc, color, label in reversed(list(zip(percentiles, colors, labels))):
    plt.plot(rs, perc, color=color, label=label)

# plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Over a Period

### Aggregate

In [ ]:
allMeans = np.array([getMeansAndStds(slicedDatas[idx])[0] for idx in range(nframes)])

In [24]:
%matplotlib ipympl

plt.xlabel("Distance from Axis")
plt.ylabel(param.title)
plt.title(f"Time-Averaged Radial Dependence of " + param.title)

plt.plot(rs, allMeans.min(axis=0), color="lightcoral", label="min mean")
plt.plot(rs, allMeans.mean(axis=0), color="blue", label="mean mean")
plt.plot(rs, allMeans.max(axis=0), color="lightcoral", label="max mean")
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …